In [1]:
import pandas as pd
from datasets import Dataset, load_dataset, Audio

In [2]:
file_names = []
transcriptions = []
for speaker in range(1, 11):
    finished_reading = False
    if speaker != 10:
        f = open(f"C:\\Users\\hojon\\OneDrive\\Desktop\\IRS Data\\TIL\\SCRIPT\\2000{speaker}0.txt", "r")
    else:
        f = open(f"C:\\Users\\hojon\\OneDrive\\Desktop\\IRS Data\\TIL\\SCRIPT\\200{speaker}0.txt", "r")

    while finished_reading != True:
        info = f.readline().split("\t")
        if info == ['']:
            finished_reading = True
        else:
            id, _ = info
        id = id[id.index('2'):]
        transcription = f.readline()
        file_names.append(f'C:\\Users\\hojon\\OneDrive\\Desktop\\IRS Data\\TIL\\WAVE\\{id}.wav')
        transcriptions.append(transcription.strip())





In [3]:
d = {"file_name": file_names, "transcription": transcriptions}
df = pd.DataFrame(data = d)
print(df)

                                              file_name  \
0     C:\Users\hojon\OneDrive\Desktop\IRS Data\TIL\W...   
1     C:\Users\hojon\OneDrive\Desktop\IRS Data\TIL\W...   
2     C:\Users\hojon\OneDrive\Desktop\IRS Data\TIL\W...   
3     C:\Users\hojon\OneDrive\Desktop\IRS Data\TIL\W...   
4     C:\Users\hojon\OneDrive\Desktop\IRS Data\TIL\W...   
...                                                 ...   
3533  C:\Users\hojon\OneDrive\Desktop\IRS Data\TIL\W...   
3534  C:\Users\hojon\OneDrive\Desktop\IRS Data\TIL\W...   
3535  C:\Users\hojon\OneDrive\Desktop\IRS Data\TIL\W...   
3536  C:\Users\hojon\OneDrive\Desktop\IRS Data\TIL\W...   
3537  C:\Users\hojon\OneDrive\Desktop\IRS Data\TIL\W...   

                                          transcription  
0         there were barrels of wine in the huge cellar  
1     she won a car because she was the twelfth pers...  
2     as they walked back they were shocked to see a...  
3             heavy rains caused a flood in the village  
4

In [4]:
# df.to_csv("metadata.csv")

In [5]:
audio_dataset = Dataset.from_dict({"audio": file_names, "transcription": transcriptions}).cast_column("audio", Audio())

In [6]:
audio_dataset

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 3538
})

In [7]:
audio_dataset[1000]['audio']

{'path': 'C:\\Users\\hojon\\OneDrive\\Desktop\\IRS Data\\TIL\\WAVE\\200040041.wav',
 'array': array([0.0010376 , 0.00137329, 0.00210571, ..., 0.0039978 , 0.00283813,
        0.00231934]),
 'sampling_rate': 16000}

In [8]:
audio_dataset.push_to_hub("casual/trainaudio")

Map:   0%|          | 0/1769 [00:00<?, ? examples/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1769 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
new_dataset = load_dataset("casual/national_speech_corpusv2")
new_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3538 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to C:/Users/hojon/.cache/huggingface/datasets/casual___parquet/casual--national_speech_corpusv2-96a77f2d17c10138/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 3538
    })
})

In [14]:
new_dataset['train'][0]['audio']

{'path': '200010001.wav',
 'array': array([-0.01141357, -0.00149536, -0.0017395 , ..., -0.00363159,
        -0.00271606, -0.00195312]),
 'sampling_rate': 16000}

competition code preprocessing

In [32]:
train = pd.read_csv("C:\\Users\\hojon\\OneDrive\\Desktop\\IRS Data\\TIL\\comps data\\Train.csv")
test = pd.read_csv("C:\\Users\\hojon\\OneDrive\\Desktop\\IRS Data\\TIL\\comps data\\Test_Novice_19May.csv")

In [33]:
test.head()

,path
0,evalb_00001.wav
1,evalb_00002.wav
2,evalb_00003.wav
3,evalb_00004.wav
4,evalb_00005.wav


In [26]:
def preprocess_data(df):
    # converting annotations to lowercase
    df['annotation'] = [annotation.lower() for annotation in df['annotation']]

    # changing the numbers (letter form) to digits
    temp = []
    word2num = {'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9,
                'ten': 10, 'eleven': 11, 'twelve': 12, 'thirteen': 13, 'fourteen': 14, 'fifteen': 15, 'sixteen': 16,
                'seventeen': 17, 'eighteen': 18, 'nineteen': 19, 'twenty': 20, 'thirty': 30, 'forty': 40, 'fifty': 50,
                'sixty': 60, 'seventy': 70, 'eighty': 80, 'ninety': 90, 'zero': 0}
    for annotation in df['annotation']:
        sentence_added = False
        for word in word2num.keys():
            if word in annotation and sentence_added == False:
                index = annotation.index(word)
                new_annotation = annotation[:index] + str(word2num[word]) + annotation[index + len(word):]
                temp.append(new_annotation)
                sentence_added = True

        if sentence_added == False:
            temp.append(annotation)

    df['annotation'] = temp

    # making the first letter uppercase, add full stop to the back
    temp = []
    for annotation in df['annotation']:
        new_annotation = annotation[0].upper() + annotation[1:] + "."
        temp.append(new_annotation)
    df['annotation'] = temp

    # changing the filepaths
    temp = []
    for path in df['path']:
        if type == 'train':
            new_path = "C:\\Users\\hojon\\OneDrive\\Desktop\\IRS Data\\TIL\\comps data\\Train\\Train\\audio\\" + path[6:]

        temp.append(new_path)
    df['path'] = temp

    audio_dataset = Dataset.from_dict({"audio": df['path'], "annotation": df['annotation']}).cast_column("audio", Audio())

    return audio_dataset

In [28]:
audio_dataset = preprocess_data(df)
print(audio_dataset)

Dataset({
    features: ['audio', 'annotation'],
    num_rows: 3750
})


In [31]:
audio_dataset[0]['annotation']

'She scored 9 for the quiz.'